## Import libraries

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from src.data_loader import read_csv_from_s3
from src.train import object_type_to_category, stratified_split, train_model

In [ ]:
TARGET = "AdoptionLikelihood"
NUM_FEATURES = [
    "AgeMonths",
    "WeightKg",
    "Vaccinated",
    "HealthCondition",
    "TimeInShelterDays",
    "AdoptionFee",
    "PreviousOwner",
]
CAT_FEATURES = ["PetType", "Breed", "Color", "Size"]
RANDOM_STATE = 42

## Load the data from S3

In [ ]:
bucket_name = "pet-adoption-mlops"
s3_data_path = "data"
file_name = "pet_adoption_data.csv"

data = read_csv_from_s3(bucket_name=bucket_name, file_key=f"{s3_data_path}/{file_name}")

In [ ]:
data = object_type_to_category(df=data)

In [ ]:
df_train, df_val, df_test = stratified_split(data)

In [ ]:
print(f"Train df shape: {df_train.shape}")
print(f"Validation df shape: {df_val.shape}")
print(f"Test df shape: {df_test.shape}")

In [ ]:
df_train[TARGET].value_counts(normalize=True)

In [ ]:
df_val[TARGET].value_counts(normalize=True)

In [ ]:
df_test[TARGET].value_counts(normalize=True)

In [ ]:
model = train_model(df_train=df_train, df_val=df_val)

In [ ]:
y_pred = model.predict(df_val[NUM_FEATURES + CAT_FEATURES])
y_true = df_val[TARGET]
accuracy = (y_pred == df_val[TARGET]).mean()
print(f"Accuracy: {accuracy}")